In [1]:
import os

os.chdir("../../")

In [2]:
import pandas as pd
import joblib
from src.features.build_features import (
    TopArtistTransformer,
    ReleaseDateTransformer,
    RemixOrCollabTransformer,
    WeekendTransformer,
    CausalInferenceTransformer
)
import logging

logging.basicConfig(level=logging.INFO)
pd.set_option("display.max_columns", None)

In [3]:
data = pd.read_csv('./data/raw/spotify_songs_train.csv')

In [4]:
train_test_data = joblib.load('./src/features/train_test_data.joblib')
preprocessing = joblib.load('./src/features/preprocessing.joblib')
causal_inference = joblib.load('./src/features/causal_inference_pipeline.joblib')

In [5]:
preprocessing

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('release_date',
                                                  Pipeline(steps=[('releasedatetransformer',
                                                                   ReleaseDateTransformer()),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  'track_album_release_date'),
                                                 ('release_day',
                                                  WeekendTransformer(),
                                                  'track_album_release_date'),
                                                 ('top_artist',
                                                  TopArtistTransformer(num_top_art...
                                                 ('track_name',
                                                  RemixOrCollabTransformer(),
                                                  'track_name'),
                                                 ('numerical',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['danceability', 'energy',
                                                   'loudness', 'speechiness',
                                                   'acousticness',
                                                   'instrumentalness',
                                                   'liveness', 'valence',
                                                   'tempo', 'duration_ms']),
                                                 ('key', OrdinalEncoder(),
                                                  ['key']),
                                                 ('mode', 'passthrough',
                                                  ['mode']),
                                                 ('drop', 'drop',
                                                  'track_album_release_date')]))])

In [6]:
X_train, X_test, y_train, y_test = train_test_data

In [7]:
X_train

,track_id,track_name,track_artist,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
21935,5C9SuiBZ0K8FYzypCT07JY,Too Close - Remastered,Next,5DyGKg106B5LCcjjbLzI3Z,New Jack Swing (The Ultimate Collection),2005-05-02,"Swingbeat (old skool), New Jack Swing, R&B, Hi...",3krpccUV68nBGAQbvHEZDC,r&b,new jack swing,0.831,0.436,7,-8.608,1,0.0566,0.010300,0.000000,0.1040,0.751,99.492,246600
6893,5ahbge6cAozopYL2EeJg82,Everything's Out In the Open,Johnnie Taylor,7DOgbrUVILzskkQmqNpuZE,In Control,1988-06-12,Southern Soul BBQ Mix,3nTS97zUqxJBQeiXNRs963,rap,southern hip hop,0.535,0.361,9,-16.820,0,0.0542,0.441000,0.000013,0.3170,0.442,75.795,233667
17093,4z08BpmVeZVdthD26JxkZt,Choke,Team Mati,27xwNsYFKXLZsfYnlZNMS2,Choke,2020-01-10,De Fiesta,37i9dQZF1DWVskFRGurTfg,latin,reggaeton,0.864,0.708,0,-7.897,1,0.0575,0.096900,0.002920,0.0905,0.716,125.027,168720
29547,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.522,0.786,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.400,128.041,353120
12791,3qel23hQWhHT3qGewBQ0pE,Men's Needs,The Cribs,0uf3G2hMRqZ0LOqv6QhZiL,"Men's Needs, Women's Needs, Whatever (Standard...",2007-05-12,Permanent Wave,5glAD13obyL0G9SH9ukBz2,rock,permanent wave,0.548,0.940,1,-2.274,0,0.0472,0.000106,0.000064,0.0696,0.783,121.994,195698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23142,4CWy5RH0DiSJtOUCcqKjbX,Trouble - Jengi Beats Remix,OFFAIAH,4bjSgHSwzKheXmr1KXyRs0,Trouble (Remixes Pt. 2),2016-09-23,Groovy // Funky // Neo-Soul,0JmBB9HfrzDiZoPVRdv8ns,r&b,neo soul,0.704,0.613,1,-7.738,0,0.0435,0.387000,0.702000,0.1160,0.484,116.032,174640
5579,0D21XvHcVsIvJM6FcGY2BT,OOOUUU,Young M.A,0o7QxqeqhALehCfm0kpJJo,OOOUUU,2016-05-20,Hip Hop Controller,37i9dQZF1DWT5MrZnPU1zD,rap,hip hop,0.640,0.819,7,-4.108,1,0.4550,0.533000,0.000000,0.1560,0.623,166.012,234526
886,6gjJnkvucgBtEyGkKQVBd0,SO AM I,WHYLO,49b0PyJsa4hXzYSGy9i2dc,SO AM I,2018-06-22,Pop Dance Hits,7eERKCkgNKCCx5GYdeMIZf,pop,dance pop,0.770,0.796,1,-5.161,0,0.0372,0.151000,0.000000,0.0837,0.308,101.985,183528
17045,1ZE19OuOnqsMFB6V9Gtzcy,Morena - Re-Mix,Hector & Tito,3syIceuWeZKFbuVubOsS4W,La Historia Live,2003-01-01,Reggaeton Classics,37i9dQZF1DX8SfyqmSFDwe,latin,reggaeton,0.811,0.750,4,-5.338,0,0.0591,0.204000,0.000000,0.0875,0.742,93.991,158373


In [8]:
X_train_fit = pd.DataFrame(preprocessing.fit_transform(X_train), columns=preprocessing.get_feature_names_out())
X_train_fit

,release_date__month_season_Fall,release_date__month_season_Spring,release_date__month_season_Summer,release_date__month_season_Winter,release_date__day_category_First 10,release_date__day_category_Last 10,release_date__day_category_Middle 10,release_date__day_category_nan,release_day__is_weekend,top_artist__is_top_artist,genres__playlist_genre_edm,genres__playlist_genre_latin,genres__playlist_genre_pop,genres__playlist_genre_r&b,genres__playlist_genre_rap,genres__playlist_genre_rock,genres__playlist_subgenre_album rock,genres__playlist_subgenre_big room,genres__playlist_subgenre_classic rock,genres__playlist_subgenre_dance pop,genres__playlist_subgenre_electro house,genres__playlist_subgenre_electropop,genres__playlist_subgenre_gangster rap,genres__playlist_subgenre_hard rock,genres__playlist_subgenre_hip hop,genres__playlist_subgenre_hip pop,genres__playlist_subgenre_indie poptimism,genres__playlist_subgenre_latin hip hop,genres__playlist_subgenre_latin pop,genres__playlist_subgenre_neo soul,genres__playlist_subgenre_new jack swing,genres__playlist_subgenre_permanent wave,genres__playlist_subgenre_pop edm,genres__playlist_subgenre_post-teen pop,genres__playlist_subgenre_progressive electro house,genres__playlist_subgenre_reggaeton,genres__playlist_subgenre_southern hip hop,genres__playlist_subgenre_trap,genres__playlist_subgenre_tropical,genres__playlist_subgenre_urban contemporary,track_name__is_remix,track_name__is_collab,numerical__danceability,numerical__energy,numerical__loudness,numerical__speechiness,numerical__acousticness,numerical__instrumentalness,numerical__liveness,numerical__valence,numerical__tempo,numerical__duration_ms,key__key,mode__mode
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.205092,-1.440577,-0.655841,-0.504629,-0.754311,-0.383001,-0.555452,1.056443,-0.794418,0.388206,7.0,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.852514,-1.853155,-3.415077,-0.528176,1.190121,-0.382946,0.830956,-0.271435,-1.677819,0.168817,9.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.434488,0.055704,-0.416945,-0.495798,-0.363348,-0.370209,-0.643323,0.906037,0.157502,-0.932907,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.942881,0.484785,0.737217,-0.647877,-0.793091,-0.364294,1.208476,-0.451923,0.269861,2.195151,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.762146,1.331944,1.472386,-0.596857,-0.800333,-0.382719,-0.779360,1.193958,0.044435,-0.475267,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22264,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.322268,-0.466894,-0.363521,-0.633159,0.946333,2.692479,-0.477344,-0.090947,-0.177823,-0.832483,1.0,0.0
22265,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.122620,0.666319,0.856161,3.404277,1.605463,-0.383001,-0.216986,0.506384,1.685383,0.183389,7.0,1.0
22266,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [9]:
causal_inference.fit_transform(data)

,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,release_year
0,Ed Sheeran,66,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,3.25,2019.0
3,The Chainsmokers,60,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,2.82,2019.0
4,Lewis Capaldi,69,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,3.15,2019.0
5,Ed Sheeran,67,pop,dance pop,0.675,0.919,8,-5.385,1,0.1270,0.079900,0.000000,0.1430,0.5850,124.982,2.72,2019.0
8,Shawn Mendes,67,pop,dance pop,0.642,0.818,2,-4.552,1,0.0320,0.056700,0.000000,0.0919,0.5900,124.957,4.22,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29545,Ferry Corsten feat. Jenny Wahlstrom,27,edm,progressive electro house,0.581,0.640,5,-8.367,1,0.0365,0.026600,0.000000,0.5720,0.2880,128.001,3.28,NaN
29546,Lush & Simon,42,edm,progressive electro house,0.428,0.922,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,3.41,2014.0
29547,Tegan and Sara,20,edm,progressive electro house,0.522,0.786,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,5.89,2013.0
29548,Starkillers,14,edm,progressive electro house,0.529,0.821,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,3.50,2014.0


In [10]:
causal_inference.fit_transform(data)

,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,release_year
0,Ed Sheeran,66,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,3.25,2019.0
3,The Chainsmokers,60,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,2.82,2019.0
4,Lewis Capaldi,69,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,3.15,2019.0
5,Ed Sheeran,67,pop,dance pop,0.675,0.919,8,-5.385,1,0.1270,0.079900,0.000000,0.1430,0.5850,124.982,2.72,2019.0
8,Shawn Mendes,67,pop,dance pop,0.642,0.818,2,-4.552,1,0.0320,0.056700,0.000000,0.0919,0.5900,124.957,4.22,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29545,Ferry Corsten feat. Jenny Wahlstrom,27,edm,progressive electro house,0.581,0.640,5,-8.367,1,0.0365,0.026600,0.000000,0.5720,0.2880,128.001,3.28,NaN
29546,Lush & Simon,42,edm,progressive electro house,0.428,0.922,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,3.41,2014.0
29547,Tegan and Sara,20,edm,progressive electro house,0.522,0.786,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,5.89,2013.0
29548,Starkillers,14,edm,progressive electro house,0.529,0.821,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,3.50,2014.0
